<a href="https://colab.research.google.com/github/jahanvi513/Pytorch/blob/main/Rice_Type_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jahanvi513
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 125MB/s]

In [21]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
data = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
data.dropna(inplace = True)
data.drop({'id'}, axis=1, inplace = True)
print(data.shape)

(18185, 11)


In [6]:
data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [7]:
original = data.copy()

for column in data.columns:
  data[column] = data[column]/data[column].abs().max()

data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [8]:
x = np.array(data.iloc[:, :-1])
y = np.array(data.iloc[:, -1])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)
x_test, x_val, y_test, y_val = train_test_split(x_test,y_test, test_size = 0.5)

In [10]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(12729, 10)
(2728, 10)
(2728, 10)


In [11]:
class dataset(Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x, dtype = torch.float32).to(device)
    self.y = torch.tensor(y, dtype = torch.float32).to(device)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, index):
    return self.x[index], self.y[index]

In [13]:
training_data = dataset(x_train, y_train)
validation_data = dataset(x_val, y_val)
testing_data = dataset(x_test, y_test)

In [23]:
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size = 8, shuffle=True)
testing_dataloader = DataLoader(testing_data, batch_size = 8, shuffle=True)

In [24]:
for x, y in train_dataloader:
  print(x)
  print("======")
  print(y)
  break

tensor([[0.8394, 0.8772, 0.8279, 0.9362, 0.7929, 0.9162, 0.6219, 0.7398, 0.8412,
         0.6011],
        [0.6326, 0.8599, 0.6492, 0.9727, 0.5973, 0.7954, 0.5343, 0.6900, 0.7288,
         0.7515],
        [0.5708, 0.8061, 0.6195, 0.9704, 0.5429, 0.7555, 0.5444, 0.6504, 0.7400,
         0.7382],
        [0.6353, 0.8971, 0.6227, 0.9825, 0.6081, 0.7970, 0.5908, 0.7097, 0.6917,
         0.8173],
        [0.7270, 0.8253, 0.7680, 0.9391, 0.7028, 0.8527, 0.6524, 0.7194, 0.7705,
         0.6097],
        [0.3548, 0.6570, 0.4766, 0.9776, 0.3436, 0.5957, 0.6081, 0.5249, 0.7063,
         0.7822],
        [0.8091, 0.8356, 0.8444, 0.9209, 0.7683, 0.8995, 0.7106, 0.7177, 0.8614,
         0.5615],
        [0.7347, 0.8084, 0.7911, 0.9284, 0.6954, 0.8571, 0.7938, 0.6872, 0.8531,
         0.5798]], device='cuda:0')
tensor([0., 1., 1., 1., 0., 1., 0., 0.], device='cuda:0')


In [26]:
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    self.input_layer = nn.Linear(x.shape[1], HIDDEN_NEURONS)
    self.linear = nn.Linear(HIDDEN_NEURONS, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = self.input_layer(x)
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

model = MyModel().to(device)

In [27]:
summary(model, (x.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [28]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 1e-3)

In [43]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_accuracy_train_plot = []
total_loss_validation_plot = []

epochs = 10
for epoch in range(epochs):
  total_accuracy_train = 0
  total_loss_train = 0
  total_accuracy_validation = 0
  total_loss_validation = 0

  for data in train_dataloader:
    inputs, labels = data
    prediction = model(inputs).squeeze(1)
    batch_loss = criterion(prediction, labels)
    total_loss_train += batch_loss.item()

    accuracy = ((prediction).round() == labels).sum().item()
    total_accuracy_train += accuracy

    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()


  break

  #model.train()

3
